<a href="https://colab.research.google.com/github/microprediction/optimizer-notebooks/blob/main/hebo_hello.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install hebo
!pip install humpday

In [3]:
from hebo.design_space.design_space import DesignSpace
from hebo.optimizers.hebo import HEBO
import pandas as pd
import numpy  as np

In [ ]:
from humpday.objectives.classic import shekel_on_cube

https://pypi.org/project/HEBO/
First optimizer I've seen to use pandas in the interface. I refrain from comment. The main thing is it won NeurIPS 2020 BBO! 


In [15]:
import math
def hebo_cube_factory(objective, n_trials, n_dim, with_count,n_suggestions=5):
    global feval_count
    feval_count = 0

    variables = [{'name': 'u' + str(i), 'type': 'num', 'lb': 0., 'ub': 1.} for i in range(n_dim)]
    space = DesignSpace().parse(variables)
    opt = HEBO(space)

    def _objective(params: pd.DataFrame) -> np.ndarray:
        global feval_count
        feval_count += len(params.index)
        return np.array([ objective(ui) for ui in params.values ])

    n_batches = int(math.floor(n_trials/n_suggestions))
    n_remainder = n_trials - n_suggestions*n_batches
    for i in range(n_batches):
        rec = opt.suggest(n_suggestions=n_suggestions) # <-- don't change this
        opt.observe(rec, _objective(rec))
    for i in range(n_remainder):
        rec = opt.suggest(n_suggestions=1)  # <-- don't change this
        opt.observe(rec, _objective(rec))

    best_val = opt.y.min()
    best_ndx = np.argmin([y[0] for y in opt.y])  # I mean seriously, why make the user do this?
    best_x = list(opt.X.values[best_ndx])
    return (best_val, best_x, feval_count) if with_count else (best_val, best_x)


In [16]:
hebo_cube_factory(shekel_on_cube,n_dim=2,n_trials=50,with_count=True, n_suggestions=15)

(-0.6556789383463781, [0.4904228753751097, 0.48570053602045316], 50)